# Creación Dimensión Proveedores

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, current_timestamp

# Lectura directa desde Silver
df_silver_prov = spark.read.table("workspace.silver.proveedores").dropDuplicates(["ruc"])

window_spec_prov = Window.orderBy("ruc")

dim_proveedores = df_silver_prov.withColumn("proveedor_key", row_number().over(window_spec_prov)) \
    .select(
        col("proveedor_key"),
        col("ruc").alias("ruc_bk"),
        col("razon_social"),
        col("direccion"),
        col("tipo_contribuyente"),
        col("forma_pago"),
        current_timestamp().alias("fecha_carga")
    )

dim_proveedores.write.mode("overwrite").saveAsTable("workspace.gold.dim_proveedores")
print("✅ dim_proveedores creada.")